In [1]:
import sys
sys.path.append("../src")

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import optuna
import warnings
from captum.attr._core.lime import get_exp_kernel_similarity_function
from captum._utils.models.linear_model import SkLearnLasso
from compare_docano_XAI import Hyper_optimalization,Check_docano_XAI,Visualization_opt,Compare_docano_XAI
from explain import STS_ExplainWrapper,SentenceTransformerToHF
from dataset import OurDataset
import architecture
import torch

In [25]:
# methods=['GAE_Explain','Occlusion','Input X Gradient','Guided Backprop','Feature Ablation','Kernel Shap','Gradient Shap','Lime','Saliency','ConservativeLRP']
# normalizations=['mean_var_normalize', 'second_moment_normalize', 'log_normalize','tanh_normalize', 'min_max_normalize', 'log_scale_normalize','log_min_max_normalize','log_mean_normalize','without_normalize']
normalizations=['without_normalize']
methods=['Gradient Shap']


In [5]:
# (0.1,0.9,{'step':0.1}) 

In [ ]:
# F_m: LIME, Feature Ablation, Kernel Shap

In [ ]:
model_param={
'Lime': {
    'similarity_func':
    {
    "function_name": ["captum.attr._core.lime.get_exp_kernel_similarity_function" ],
    "parameters": {"distance_mode": ['cosine',"euclidean"], "kernel_width": [450,750]},
    },
    'interpretable_model':{
    "function_name": ["captum._utils.models.linear_model.SkLearnLasso"],# ,"captum._utils.models.linear_model.LinearModel"
    "parameters": {"alpha": [1e-19,1e-25]},
}
},
'LayerGradientXActivation':{"layer":[["model.get_embedding_layer"]],'multiply_by_inputs':True},
'GAE_Explain':{'implemented_method':True,'layers':{'module_path_expressions':["hf_transformer.encoder.layer.*.attention.self.dropout"]}},
'ConservativeLRP':{'implemented_method':True,'layers':{'store_A_path_expressions':["hf_transformer.embeddings"],'attent_path_expressions':['hf_transformer.encoder.layer.*.attention.self.dropout'],'norm_layer_path_expressions':["hf_transformer.embeddings.LayerNorm","hf_transformer.encoder.layer.*.attention.output.LayerNorm","hf_transformer.encoder.layer.*.output.LayerNorm"]}},
'Occlusion_word_level':{'implemented_method':True}
}
method_param={
'Lime':{"parameters":{"n_samples":[80,90]}, "token_groups_for_feature_mask": True},    ### Lime
'Saliency':{"parameters":{'abs':[True,False]}},                                                      ### Saliency    
'Occlusion':{"parameters":{"sliding_window_shapes":[(3,1024),(5,1024)],'strides':[(1,1024),(1,512)]},"compute_baseline": True},  ### Occlusion,  ,(5,1024)
'Input X Gradient':{},                                                                            ### Input X Gradient
'Guided Backprop':{},                                                                            ### Guided Backprop
'Deconvolution':{},                                                                            ### Deconvolution
'Gradient Shap':{"parameters":{'stdevs':(0.1,0.9,{'step':0.1}) ,'n_samples':[10,15]},"compute_baseline": True },   ### GradientShap # 
'Kernel Shap':{"parameters":{'n_samples':[80,90]},"token_groups_for_feature_mask": True},          ### KernelShap
'Feature Ablation':{"token_groups_for_feature_mask": True ,"compute_baseline": True},
'GAE_Explain':{},
'ConservativeLRP':{},
'Occlusion_word_level':{"parameters":{'regex_condition':['',".,!?;:…"]}}, # If you want only word put empty string  ''   
'Integrated Gradients':{"parameters":{'n_steps':[60,40]}},
'LayerGradientXActivation':{},                                                   ### LayerXGradient
}          


In [6]:
combinations = Hyper_optimalization.compute_combinations(model_param, method_param, methods, normalizations)
print(len(combinations))

4


In [11]:
dataset = OurDataset(csv_dirpath='./data')

In [27]:
hyper_opt_exp = Hyper_optimalization(rationale_path=f"../data/annotations/rationale.json",
                                     explanations_path="./results/.json",
                                     dataset=dataset,
                                     perc=100,
                                     model_path= "intfloat/multilingual-e5-large",#"../models/GTR-T5-FT",   
                                     embeddings_module_name='embeddings.word_embeddings',#,#encoder.embed_tokens
                                     methods=methods,
                                     normalizations=normalizations,
                                     explanation_maps_sentence = True,
                                     model_param=model_param,
                                     method_param=method_param,
                                     plausability_weight=0.5,
                                     faithfulness_weight=0.5,
                                     multiple_object=False,
                                    )
print(f'length of dataset: {len(hyper_opt_exp.dataset)}')

length of dataset: 16


In [28]:
# available samplers: TPESampler,NSGAIISampler,NSGAIIISampler,BruteForceSampler,GPSampler 
sampler='BruteForceSampler'
best,trials,Visualization_opt=hyper_opt_exp.run_optimization(sampler=sampler,n_trials=1) # ,n_startup_trials=4,seed=1000
Visualization_opt.table_counter(save_path_plot='./results/count_tables/baseline_counter.csv') 
Visualization_opt.table_sampler(save_path_plot=f'./results/sampler_tables/baseline_sampler.csv')
Visualization_opt.visual_aopc(save_path_plot=f'./results/aopc_plots/')

[I 2025-07-03 16:51:20,223] A new study created in memory with name: no-name-b5153e82-85fa-458f-be11-430c1fd05699


{'stdevs': 0.2, 'n_samples': 15}
without_normalize
Gradient Shap


[W 2025-07-03 16:51:25,726] Trial 0 failed with parameters: {'method': 'Gradient Shap', 'normalization': 'without_normalize', 'Gradient Shap-stdevs': 0.2, 'Gradient Shap-n_samples': 15} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\Dell\anaconda3\envs\env_autoxai_project\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "c:\Users\Dell\Desktop\Kinit\o-mega\notebooks\../src\compare_docano_XAI.py", line 1768, in objective
    with warnings.catch_warnings():
  File "c:\Users\Dell\Desktop\Kinit\o-mega\notebooks\../src\compare_docano_XAI.py", line 1477, in compute_explanation
    if method in model_param:
  File "c:\Users\Dell\Desktop\Kinit\o-mega\notebooks\../src\compare_docano_XAI.py", line 1391, in set_model
  File "c:\Users\Dell\Desktop\Kinit\o-mega\notebooks\../src\explain.py", line 72, in setup_transformer
    transformer = SentenceTransformerToHF(
  File "c:\Users\De

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Dell\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Dell\AppData\Local\Temp\ipykernel_6468\1527671779.py", line 3, in <module>
    best,trials,Visualization_opt=hyper_opt_exp.run_optimization(sampler=sampler,n_trials=1) # ,n_startup_trials=4,seed=1000
  File "c:\Users\Dell\Desktop\Kinit\o-mega\notebooks\../src\compare_docano_XAI.py", line 1920, in run_optimization
    else:
  File "c:\Users\Dell\anaconda3\envs\env_autoxai_project\lib\site-packages\optuna\study\study.py", line 475, in optimize
    _optimize(
  File "c:\Users\Dell\anaconda3\envs\env_autoxai_project\lib\site-packages\optuna\study\_optimize.py", line 63, in _optimize
    _optimize_sequential(
  File "c:\Users\Dell\anaconda3\envs\env_autoxai_project\lib\site-packages\optuna\study\_optimize.py", line 160, in _optimize_sequential
    frozen_trial = 